In [1]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
import sys

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

In [21]:
categorical_col = ['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'timestamp', 'minute', 'second']
target = 'is_attributed'
feature_cols = list(set(train.columns) - set([target]))

from sklearn.model_selection import train_test_split
trainset, valset = train_test_split(train,test_size=0.5, random_state=42)



In [22]:
y_train = trainset[target].values
y_val = valset[target].values

lgb_train = lgb.Dataset(trainset[feature_cols], y_train, categorical_feature = categorical_col)
lgb_val = lgb.Dataset(valset[feature_cols], y_val, categorical_feature = categorical_col)

import gc
gc.collect()

850

In [23]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 30,
        'learning_rate': 0.1,
        'num_leaves': 61,
        'num_threads': 6, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 300,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.8, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'bagging_fraction': 0.8, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'metric' : ['auc', 'binary_logloss']
    }

model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[10]	valid_0's binary_logloss: 0.201217	valid_0's auc: 0.942605


KeyboardInterrupt: 

In [10]:
bb = model.predict(test)

In [11]:
bb

array([ 0.92762108,  0.92762108,  0.92762108, ...,  0.92762108,
        0.92762108,  0.92762108])